In [1]:
import os
import sys
import time
import argparse
import json
from tqdm import tqdm

import pandas as pd
import numpy as np
import pykeen
from pykeen.evaluation import rank_based_evaluator
import torch
from scipy import stats

from sheaf_kg.train_sheafE_betae import read_dataset, shuffle_datasets, dataset_to_device, sample_answers
import sheaf_kg.complex_functions as cf
# from sheaf_kg.complex_functions import test_batch

PyKEEN was unable to load dataset openbiolinkf1. Try uninstalling PyKEEN with ``pip uninstall pykeen`` then reinstalling
PyKEEN was unable to load dataset openbiolinkf2. Try uninstalling PyKEEN with ``pip uninstall pykeen`` then reinstalling


In [2]:
dataset = 'FB15k-237'
savename = 'TransE_1000epochs_64embdim_SoftplusLossloss_1234seed_20210519-1452'

savedir = f'/home/gebhart/projects/sheaf_kg/data/{dataset}'
complex_dataset_loc = f'/home/gebhart/projects/sheaf_kg/data/{dataset}-betae'

test_query_structures = ['1p','2p','3p','2i','3i','ip','pi']


In [3]:
model = torch.load(os.path.join(savedir, savename, 'trained_model.pkl'))

In [4]:
fun_map = {'1p':cf.L_p_traversal_transE,
           '2p':cf.L_p_traversal_transE,
           '3p':cf.L_p_traversal_transE,
           '2i':cf.L_i_traversal_transE,
           '3i':cf.L_i_traversal_transE,
           'ip':cf.L_ip_traversal_transE,
           'pi':cf.L_pi_traversal_transE
          }

def test_batch(model, test_data, model_inverses=False, test_batch_size=5,
                test_query_structures=test_query_structures,
                ks=[1,3,5,10]):
    with torch.no_grad():
        results = []
        for query_structure in test_query_structures:
            print('Running query : {}'.format(query_structure))
            all_avg_ranks = []
            cnt = 0
            num_test = len(test_data[query_structure]['answers'])
            for qix in tqdm(range(0, num_test//2, test_batch_size)):
                if num_test - qix == 1:
                    continue
                entities = test_data[query_structure]['entities'][qix:qix+test_batch_size]
                relations = test_data[query_structure]['relations'][qix:qix+test_batch_size]
                if model_inverses:
                    inverses = None
                else:
                    inverses = test_data[query_structure]['inverses'][qix:qix+test_batch_size]
                all_answers = test_data[query_structure]['answers'][qix:qix+test_batch_size]
                targets = torch.arange(model.entity_embeddings(indices=None).shape[0]).to(model.device)
                Q = fun_map[query_structure](model, entities, relations, targets, invs=inverses, p=1)
                answer_lens = np.array([len(a) for a in all_answers])
                max_len = answer_lens.max()
                for l in np.unique(answer_lens):
                    idxs = np.where(answer_lens == l)[0]
                    answers = [all_answers[j] for j in idxs]
                    filter_fix = torch.arange(l)
                    for aix in range(len(idxs)):
                        ranks = rank_based_evaluator.compute_rank_from_scores(Q[idxs[aix],answers[aix]].unsqueeze(1), Q[idxs[aix],:].unsqueeze(0))
                        avg_rank = (torch.sort(ranks['realistic'].cpu(), dim=0)[0] - filter_fix).numpy()
                        all_avg_ranks.append(avg_rank)
                        print(avg_rank.shape)
            all_avg_ranks = np.concatenate(all_avg_ranks)
            rd = {k: np.mean(all_avg_ranks <= k) for k in ks}
            mrr = np.reciprocal(stats.hmean(all_avg_ranks))
            rd['mrr'] = mrr if isinstance(mrr, float) else mrr[0]
            # rd['mr'] = np.mean(all_avg_ranks)
            results.append(rd)

        df = pd.DataFrame(results, index=test_query_structures)
        return df

In [5]:
datasets = read_dataset(complex_dataset_loc)
datasets = dataset_to_device(shuffle_datasets(datasets), model.device)

In [6]:
extension_df = test_batch(model, datasets['test-easy'], model_inverses=True, test_query_structures=test_query_structures)
extension_df*100

  2%|▏         | 37/1631 [00:00<00:04, 364.25it/s]

Running query : 1p
(3,)
(5,)
(6,)
(18,)
(23,)
(2,)
(2,)
(3,)
(5,)
(54,)
(1,)
(3,)
(6,)
(21,)
(23,)
(1,)
(4,)
(5,)
(11,)
(326,)
(1,)
(1,)
(5,)
(7,)
(35,)
(4,)
(17,)
(18,)
(84,)
(108,)
(4,)
(4,)
(4,)
(6,)
(11,)
(4,)
(8,)
(12,)
(13,)
(15,)
(1,)
(6,)
(9,)
(10,)
(11,)
(1,)
(5,)
(7,)
(8,)
(12,)
(2,)
(4,)
(6,)
(9,)
(15,)
(3,)
(3,)
(5,)
(9,)
(34,)
(1,)
(2,)
(2,)
(21,)
(37,)
(2,)
(2,)
(5,)
(9,)
(158,)
(1,)
(3,)
(3,)
(4,)
(47,)
(1,)
(1,)
(3,)
(3,)
(4,)
(1,)
(1,)
(2,)
(8,)
(10,)
(1,)
(3,)
(6,)
(8,)
(38,)
(5,)
(7,)
(7,)
(13,)
(35,)
(2,)
(2,)
(3,)
(13,)
(81,)
(1,)
(3,)
(6,)
(10,)
(14,)
(4,)
(8,)
(9,)
(11,)
(74,)
(1,)
(4,)
(5,)
(33,)
(64,)
(1,)
(2,)
(6,)
(23,)
(32,)
(2,)
(3,)
(4,)
(7,)
(9,)
(1,)
(10,)
(11,)
(12,)
(12,)
(2,)
(3,)
(5,)
(5,)
(337,)
(2,)
(7,)
(8,)
(35,)
(48,)
(1,)
(2,)
(3,)
(10,)
(19,)
(3,)
(5,)
(5,)
(12,)
(36,)
(2,)
(3,)
(11,)
(67,)
(105,)
(1,)
(1,)
(5,)
(10,)
(15,)
(1,)
(2,)
(2,)
(2,)
(4,)
(1,)
(1,)
(2,)
(2,)
(2,)
(2,)
(2,)
(2,)
(3,)
(6,)
(1,)
(2,)
(5,)
(5,)
(6,)
(5,)
(7,)
(21,)
(27,)

  7%|▋         | 114/1631 [00:00<00:04, 358.26it/s]

(1,)
(4,)
(9,)
(21,)
(25,)
(1,)
(2,)
(3,)
(3,)
(6,)
(1,)
(5,)
(5,)
(8,)
(8,)
(1,)
(7,)
(9,)
(125,)
(376,)
(2,)
(3,)
(13,)
(37,)
(67,)
(3,)
(7,)
(11,)
(13,)
(54,)
(3,)
(4,)
(4,)
(8,)
(28,)
(2,)
(2,)
(10,)
(13,)
(15,)
(1,)
(2,)
(3,)
(5,)
(14,)
(5,)
(8,)
(10,)
(17,)
(23,)
(2,)
(5,)
(5,)
(7,)
(12,)
(1,)
(1,)
(1,)
(7,)
(21,)
(3,)
(4,)
(7,)
(13,)
(20,)
(2,)
(5,)
(8,)
(10,)
(37,)
(1,)
(2,)
(2,)
(5,)
(42,)
(1,)
(2,)
(4,)
(25,)
(55,)
(3,)
(3,)
(4,)
(5,)
(5,)
(5,)
(5,)
(9,)
(15,)
(18,)
(1,)
(1,)
(2,)
(2,)
(3,)
(1,)
(2,)
(7,)
(7,)
(46,)
(2,)
(2,)
(7,)
(7,)
(79,)
(1,)
(1,)
(4,)
(6,)
(13,)
(4,)
(5,)
(6,)
(10,)
(51,)
(2,)
(3,)
(4,)
(13,)
(17,)
(1,)
(7,)
(7,)
(13,)
(110,)
(1,)
(2,)
(5,)
(17,)
(21,)
(4,)
(8,)
(14,)
(20,)
(34,)
(2,)
(3,)
(5,)
(5,)
(95,)
(1,)
(2,)
(3,)
(6,)
(23,)
(1,)
(1,)
(3,)
(8,)
(18,)
(1,)
(1,)
(2,)
(4,)
(19,)
(1,)
(2,)
(5,)
(17,)
(80,)
(2,)
(3,)
(3,)
(16,)
(22,)
(2,)
(3,)
(3,)
(10,)
(15,)
(2,)
(2,)
(3,)
(11,)
(13,)
(2,)
(2,)
(4,)
(5,)
(5,)
(1,)
(3,)
(11,)
(16,)
(17,)
(1,)
(1,)
(2,)

 12%|█▏        | 190/1631 [00:00<00:03, 367.10it/s]

...

  7%|▋         | 29/429 [00:00<00:02, 143.20it/s]

(1,)
(18,)
(71,)
(107,)
(245,)
(3,)
(3,)
(16,)
(145,)
(491,)
(2,)
(8,)
(11,)
(20,)
(54,)
(1,)
(6,)
(14,)
(266,)
(600,)
(4,)
(5,)
(56,)
(153,)
(272,)
(8,)
(12,)
(16,)
(20,)
(186,)
(4,)
(16,)
(41,)
(83,)
(140,)
(1,)
(4,)
(4,)
(12,)
(62,)
(4,)
(7,)
(15,)
(47,)
(106,)
(8,)
(9,)
(13,)
(30,)
(110,)
(10,)
(14,)
(21,)
(317,)
(583,)
(6,)
(9,)
(12,)
(14,)
(573,)
(4,)
(6,)
(24,)
(32,)
(142,)
(8,)
(11,)
(12,)
(15,)
(261,)
(4,)
(4,)
(10,)
(49,)
(87,)
(2,)
(7,)
(10,)
(11,)
(68,)
(1,)
(4,)
(22,)
(65,)
(150,)
(6,)
(14,)
(20,)
(29,)
(596,)
(9,)
(14,)
(16,)
(41,)
(582,)
(1,)
(2,)
(2,)
(2,)
(13,)
(22,)
(68,)
(207,)
(580,)
(882,)
(1,)
(24,)
(25,)
(274,)
(573,)
(3,)
(5,)
(8,)
(9,)
(35,)
(11,)
(35,)
(98,)
(156,)
(213,)
(1,)
(2,)
(3,)
(69,)
(89,)
(4,)
(8,)
(11,)
(467,)
(568,)
(2,)
(9,)
(99,)
(167,)
(600,)
(13,)
(85,)
(100,)
(488,)
(569,)
(5,)


  9%|▉         | 39/429 [00:00<00:02, 133.12it/s]


(10,)
(17,)
(31,)
(110,)
(9,)
(16,)
(19,)
(302,)
(313,)
(4,)
(5,)
(72,)
(266,)
(669,)
(7,)
(12,)
(12,)
(47,)
(231,)
(2,)
(2,)
(8,)
(8,)
(58,)
(1,)
(9,)
(9,)
(10,)
(443,)
(1,)
(8,)
(49,)
(59,)
(199,)
(3,)
(14,)
(46,)
(52,)
(461,)
(10,)
(17,)
(24,)
(42,)
(87,)
(1,)
(56,)
(183,)
(569,)
(584,)
(3,)
(5,)
(79,)


KeyboardInterrupt: 